In [13]:
import numpy as np
import pandas as pd
from pynwb import NWBHDF5IO
from tqdm import tqdm  # For showing a progress bar

In [14]:
filepath = './sub-9_ses-mouse-9-session-date-2017-08-19-area-visA-L23-multi-plane-imaging_behavior+ophys.nwb'

In [15]:
# read and examine the NWB file
io = NWBHDF5IO(filepath, 'r')
nwb = io.read()

planeSeg_0_df = nwb.processing['ophys']['ImageSegmentation']['PlaneSegmentation_0'].to_dataframe()
ml_0 = planeSeg_0_df['ml'].values
n_neurons_0 = ml_0.shape[0]
print("Number of neurons: ", n_neurons_0)
# extract data from deconvolved_activity_plane_0 and 
imaging_timestamps_0 = nwb.processing['ophys']['deconvolved_activity_plane_0'].timestamps
print("Number of timestamps:" , len(imaging_timestamps_0))

/Users/amandalmia/workspace/pharam/venv/lib/python3.8/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/Users/amandalmia/workspace/pharam/venv/lib/python3.8/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.3.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)


Number of neurons:  224
Number of timestamps: 32496


In [16]:
#pre processing 
imaging_timestamps_0 = nwb.processing['ophys']['deconvolved_activity_plane_0'].timestamps
imaging_frame_timestamps = nwb.processing['behavior']['frame_aligned_position']['frame_aligned_forward_and_lateral_position'].timestamps
frame_aligned_position = nwb.processing['behavior']['frame_aligned_position']['frame_aligned_forward_and_lateral_position'].data[:]
imaging_timestamps_0_set = set(imaging_timestamps_0)
common_timestamps_indexes = [i for i, timestamp in enumerate(imaging_frame_timestamps) if timestamp in imaging_timestamps_0_set]
filtered_frame_aligned_position = [frame_aligned_position[i] for i in common_timestamps_indexes]
positions = np.vstack(filtered_frame_aligned_position)

In [17]:
imaging_activity_0 = nwb.processing['ophys']['deconvolved_activity_plane_0'].data[:]
imaging_activity_1 = nwb.processing['ophys']['deconvolved_activity_plane_1'].data[:]
imaging_activity_2 = nwb.processing['ophys']['deconvolved_activity_plane_2'].data[:]
imaging_activity_3 = nwb.processing['ophys']['deconvolved_activity_plane_3'].data[:]

df_imaging_0 = pd.DataFrame(imaging_activity_0)
df_imaging_0.columns = ['{}_plane_0'.format(col) for col in df_imaging_0.columns]
df_imaging_1 = pd.DataFrame(imaging_activity_1)
df_imaging_1.columns = ['{}_plane_1'.format(col) for col in df_imaging_1.columns]
df_imaging_2 = pd.DataFrame(imaging_activity_2)
df_imaging_2.columns = ['{}_plane_2'.format(col) for col in df_imaging_2.columns]
df_imaging_3 = pd.DataFrame(imaging_activity_3)
df_imaging_3.columns = ['{}_plane_3'.format(col) for col in df_imaging_3.columns]

df_imaging = pd.concat([df_imaging_0, df_imaging_1,df_imaging_2,df_imaging_3], axis=1)


df_imaging['ruleAorRuleB'] = -1
df_imaging['WrongRightDecision'] = -1
df_imaging['trial'] = 0

df_imaging['Position_X'] = positions[:, 0]  # Add X positions as a new column
df_imaging['Position_Y'] = positions[:, 1]  # Add Y positions as a new column

df_imaging

,0_plane_0,1_plane_0,2_plane_0,3_plane_0,4_plane_0,5_plane_0,6_plane_0,7_plane_0,8_plane_0,9_plane_0,...,112_plane_3,113_plane_3,114_plane_3,115_plane_3,116_plane_3,ruleAorRuleB,WrongRightDecision,trial,Position_X,Position_Y
0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,-1,-1,0,NaN,NaN
1,0.0,0.0,0.0,0.00000,0.101311,0.000000,0.0,0.320396,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,-1,-1,0,NaN,NaN
2,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.116361,0.000000,...,0.0,0.0,0.0,0.0,0.0,-1,-1,0,NaN,NaN
3,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.021883,0.043689,0.000000,...,0.0,0.0,0.0,0.0,0.0,-1,-1,0,NaN,NaN
4,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.134092,0.071545,0.000000,...,0.0,0.0,0.0,0.0,0.0,-1,-1,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32491,0.0,0.0,0.0,0.00000,0.000000,0.100865,0.0,0.048600,1.043518,0.000000,...,0.0,0.0,0.0,0.0,0.0,-1,-1,0,NaN,NaN
32492,0.0,0.0,0.0,0.00000,0.000000,0.203986,0.0,0.080403,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,-1,-1,0,NaN,NaN
32493,0.0,0.0,0.0,0.00000,0.069372,0.000000,0.0,0.005445,0.417581,0.000000,...,0.0,0.0,0.0,0.0,0.0,-1,-1,0,NaN,NaN
32494,0.0,0.0,0.0,3.79154,0.000000,0.000000,0.0,0.124143,5.369876,0.207785,...,0.0,0.0,0.0,0.0,0.0,-1,-1,0,NaN,NaN


In [18]:
# Iterate over the DataFrame using iterrows (note: itertuples is faster if you have a very large DataFrame)
trial = -1
prevNA= True

trial_df = nwb.trials.to_dataframe()

def isRuleA(trial):
    if trial_df.at[trial, 'is_ruleA']:
        return 1
    else:
        return -1
    
def isCorrect(trial):
    if trial_df.at[trial, 'is_correct']:
        return 1
    else:
        return -1

for index, row in tqdm(df_imaging.iterrows(), total=df_imaging.shape[0]):
    # Check if both Position_X and Position_Y are NaN
    if pd.isna(row['Position_X']) and pd.isna(row['Position_Y']):
        df_imaging.drop(index, inplace=True)
        prevNA=True
    else :
        if (prevNA):
            trial+=1
        df_imaging.at[index, 'trial'] = trial
        df_imaging.at[index, 'ruleAorRuleB'] = isRuleA(trial)
        df_imaging.at[index, 'WrongRightDecision'] = isCorrect(trial)
        prevNA=False
df_imaging.reset_index(drop=True, inplace=True)


  3%|▎         | 1074/32496 [00:06<03:04, 170.18it/s]

In [ ]:
df_imaging.to_pickle("raw-clean-mouse-9-session-date-2017-08-19.pkl")
del df_imaging
df_imaging = pd.read_pickle("raw-clean-mouse-9-session-date-2017-08-19.pkl")
df_imaging.head()

In [ ]:
# Preparatiion of the data for the LSTM model
X = list()
y = list()
window_size = 20
for group_name, group_data in df_imaging.groupby("trial"):
    for i in range(window_size, len(group_data) - 1, 1):
        X.append((group_data.iloc[i - window_size:i].to_numpy().tolist()))
        y.append([df_imaging.iloc[i - 1]["Position_X"], df_imaging.iloc[i - 1]["Position_Y"]])

X_numpy = np.array(X)
y_numpy = np.array(y)

np.save('mouse-9-session-date-2017-08-19_X.npy', X_numpy)
np.save('mouse-9-session-date-2017-08-19_y.npy', y_numpy)

In [ ]:
X_numpy.shape